TRAINING MODEL


In [1]:
import pandas as pd
import numpy as np
import torch
import os
from torchvision import transforms
from PIL import Image

In [2]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),   # Set fixed size for vgg model
    transforms.ToTensor(),           # Convert to [3, 224, 224] Tensor-> array conversion
    transforms.Normalize(            # Normalize pixel values
        mean=[0.485, 0.456, 0.406], 
        std=[0.229, 0.224, 0.225]
    )
])

In [ ]:
directory=r"mask_data"
categories=['with_mask','without_mask']
data=[]
label=[]

for category in categories:
    path=os.path.join(directory , category)
    for img in os.listdir(path):
        img_path=os.path.join(path , img)
        image=Image.open(img_path).convert('RGB') #TO PIL
        image=transform(image) 
        
        data.append(image)                 #non numeric values
        label.append(category)           #non numeric values
        
label

c:\Users\goyal\AppData\Local\Programs\Python\Python312\Lib\site-packages\PIL\Image.py:1056: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


['with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with

In [4]:
for i in range (len(label)):
    if label[i] =='with_mask':
        label[i]=1
        
    else:
        label[i]=0
label_tensor = torch.tensor(label, dtype=torch.long)
data_tensor = torch.stack(data)

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, label, test_size=0.2, random_state=42)

In [6]:
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import numpy as np
import torch

class CustomDataset(Dataset):
    def __init__(self, features, labels, transform=None):
        self.features = features
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.features)

    def __getitem__(self, index):
        return self.features[index], self.labels[index]


In [7]:
train_dataset = CustomDataset(X_train, y_train, transform=transform)
test_dataset = CustomDataset(X_test, y_test, transform=transform)

In [8]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, pin_memory=True)

In [9]:
import torchvision.models as models
vgg16 = models.vgg16(pretrained=True)

c:\Users\goyal\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\goyal\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [10]:
for param in vgg16.features.parameters():
    param.requires_grad=False

In [11]:
import torch.nn as nn
vgg16.classifier = nn.Sequential(
    nn.Linear(25088, 4096),  # VGG16's flattened input size
    nn.ReLU(),
    nn.Dropout(0.5),

    nn.Linear(4096, 1024),
    nn.ReLU(),
    nn.Dropout(0.3),

    nn.Linear(1024, 2)  # Output: 2 classes (mask/no mask)
)

In [12]:
learning_rate = 0.0001
epochs = 10

In [13]:
criterion = nn.CrossEntropyLoss()
import torch.optim as optim
optimizer = optim.Adam(vgg16.classifier.parameters(), lr=learning_rate)

In [14]:
for epoch in range (epochs):
    for batch_features, batch_labels in train_loader:
        
        output=vgg16(batch_features)
        
        loss=criterion(output,batch_labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        break

c:\Users\goyal\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


In [15]:
# evaluation on test data
vgg16.eval()#disables layers like Dropout and BatchNorm
total = 0
correct = 0

with torch.no_grad():

  for batch_features, batch_labels in test_loader:

    outputs = vgg16(batch_features)

    _, predicted = torch.max(outputs, 1)

    total = total + batch_labels.shape[0]

    correct = correct + (predicted == batch_labels).sum().item()

print(correct/total)

0.9716535433070866


COMPUTER VISION

In [16]:
import cv2

In [17]:
def detect_face_mask(img):
    vgg16.eval()

    with torch.no_grad():
        # Convert BGR to RGB and prepare PIL image
        img_pil = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        
        # Apply transforms and add batch dimension
        img_transformed = transform(img_pil).unsqueeze(0)

        # Get raw model outputs (logits)
        output = vgg16(img_transformed)  # shape: [1, 2]

        # Apply softmax to get probabilities
        probs = torch.nn.functional.softmax(output, dim=1)

        # Get probability of class "with_mask" (index 1)
        mask_prob = probs[0][1].item()  # e.g. 0.85 for 85%

        return mask_prob * 100  # Return as percentage



In [21]:
cap=cv2.VideoCapture(0)
while True:
    ret,frame=cap.read()
    ray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  # Haar works on grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_alt2.xml')
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

    for (x, y, w, h) in faces:
        # Extract the face ROI
        face_img = frame[y:y+h, x:x+w]
        try:
            # Resize to model input size (224x224)
            resized_face = cv2.resize(face_img, (224, 224))

            # Predict mask probability
            mask_percent = detect_face_mask(resized_face)

            # Set label and color
            label = f"{mask_percent:.1f}% Mask"
            color = (0, 255, 0) if mask_percent >= 50 else (0, 0, 255)

            # Draw rectangle and label
            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
            cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)

        except:
            continue  # In case resize or prediction fails

    # Show the frame
    cv2.imshow("Face Mask Detection", frame)

    # Break on 'x'
    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()


In [25]:
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Detect faces using cvlib
    faces, confidences = cv.detect_face(frame)

    for face in faces:
        (startX, startY) = face[0], face[1]
        (endX, endY) = face[2], face[3]

        # Crop face
        face_crop = frame[startY:endY, startX:endX]

        try:
            # Convert to PIL for transform
            face_pil = Image.fromarray(cv2.cvtColor(face_crop, cv2.COLOR_BGR2RGB)).convert('RGB')
            input_tensor = transform(face_pil)
            input_tensor = input_tensor.unsqueeze(0)  # Add batch dimension

            # Predict
            with torch.no_grad():
                output = vgg16(input_tensor)
                probs = torch.nn.functional.softmax(output, dim=1)
                confidence = probs[0][1].item()  # 1 = With Mask
                label = "Mask" if confidence > 0.5 else "No Mask"
                percentage = int(confidence * 100) if label == "Mask" else int((1 - confidence) * 100)

            # Set color based on prediction
            color = (0, 255, 0) if label == "Mask" else (0, 0, 255)
            text = f"{label}: {percentage}%"

            # Draw bounding box and label
            cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)
            cv2.putText(frame, text, (startX, startY - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)

        except Exception as e:
            print("Skipping one face due to error:", e)
            continue

    # Show video
    cv2.imshow("Real-Time Mask Detection", frame)

    # ESC to exit
    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()